# Separación de los datos y selección de características

## Separación de los datos

* Percentatge
* Cross validation

## Selección de características


### Métodos de filtrado (*Filter methods*)


Los métodos de selección de características de filtro aplican una medida estadística para asignar una puntuación a cada característica. Las características se clasifican según la puntuación y se seleccionan para guardarlas o eliminarlas del conjunto de datos. Los métodos son a menudo univariados y consideran la característica de forma independiente o con respecto a la variable dependiente.

### Métodos de envoltura (*wrapper methods*)

Los métodos de envoltura consideran la selección de un conjunto de características como un problema de búsqueda, donde se preparan, evalúan y comparan diferentes combinaciones con otras combinaciones. Un modelo predictivo que usamos para evaluar una combinación de características y asignar una puntuación basada en la precisión del modelo.

El proceso de búsqueda puede ser metódico, como una búsqueda de primer orden, puede ser estocástico, como un algoritmo aleatorio de escalada, o puede usar heurística, como pases hacia adelante y hacia atrás para agregar y eliminar características.

### Métodos Embebidos (*Embed Methods*)

Los métodos integrados aprenden qué características contribuyen mejor a la precisión del modelo mientras se crea el modelo. El tipo más común de métodos de selección de funciones integradas son los métodos de regularización.


#### Lecturas adicionales

[An Introduction to Feature Selection](https://machinelearningmastery.com/an-introduction-to-feature-selection/)